In [1]:
import pandas as pd
import numpy as np

## Preview data

In [2]:
busi = pd.read_csv('../data/business_data_subset.csv')
rev = pd.read_csv('../data/reviews_data_subset.csv', parse_dates=['date'])
user = pd.read_csv('../data/user_df.csv', low_memory=False)

In [3]:
# business_data
busi.head(1)

,business_id,business_name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,0,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","Food, Restaurants, Grocery, Middle Eastern",NaN


In [4]:
# review_data
rev.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,0.0,0.0,0.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52


In [5]:
# user_data
user.head(1)

,Unnamed: 0,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,2007-07-06 03:27:11,628,225,227,NaN,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...",...,2,1,0,1,11,15,22,22,10,0


## Concat review_df & business_df

In [6]:
rev_ = rev[['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']]
busi_ = busi[['business_id', 'business_name', 'city', 'state', 'categories']]

# add business info onto each review
rev_busi_ = rev_.merge(busi_, how='outer', on='business_id')
rev_busi_ = rev_busi_.reset_index(drop=True)
rev_busi_.head(1)

,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
0,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52,Cabo Mexican Restaurant,Las Vegas,NV,"Restaurants, Mexican"


In [7]:
rev_busi_.describe() # 3,000,000 reviews

,stars
count,2.982268e+06
mean,3.770697e+00
std,1.379614e+00
min,1.000000e+00
25%,3.000000e+00
50%,4.000000e+00
75%,5.000000e+00
max,5.000000e+00


## Filter based on City

In [8]:
# only maintain reviews that meets city = 'Phoenix'
rev_busi_Pho = rev_busi_[rev_busi_['city'] == 'Phoenix']
rev_busi_Pho = rev_busi_Pho.reset_index(drop=True)

rev_busi_Pho.head(1) # 500,000 reviews

,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
0,UGErdm6bt48SXTVwJIti2Q,QodunSzok4nIYFNrTSGesQ,poSV39UqEg-gpESXafS9-g,3.0,My husband and I go there once a week since la...,2018-03-04 01:03:53,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."


## Filter reviews: only keep the latest review for each user on each business

In [9]:
user_busi_groupby = rev_busi_Pho.groupby(['user_id', 'business_id'])
review_id_lst = []
for group_name, group_df in user_busi_groupby:
    group_df.sort_values(by='date')
    group_df.reset_index(inplace=True, drop=True)
    review_id_lst.append(group_df.iloc[-1,0]) # the latest review by each user on each business

In [10]:
rev_busi_Pho_ = rev_busi_Pho[rev_busi_Pho['review_id'].isin(review_id_lst)]
len(rev_busi_Pho_) 

492333

## Filter users & business based on reviews' counts

In [11]:
def review_filters(data, filtered_var, base_var, threshold):
    '''
    only keep the values of filter_var that meets:
    filter_var[base_var].sum() >= threshold
    '''
    count_df = data[[filtered_var, base_var]].groupby(filtered_var).count()
    valid_id = count_df[count_df[base_var]>=threshold].index.tolist()
    data = data[rev_busi_Pho[filtered_var].isin(valid_id)]
    return data

In [12]:
# user_id['review_id'].sum() >= 5
rev_busi_Pho = review_filters(rev_busi_Pho_, 'user_id', 'review_id', 5)

/Users/zhangwenxin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [13]:
# business_id['review_id'].sum() >= 30
rev_busi_Pho = review_filters(rev_busi_Pho, 'business_id', 'review_id', 30)

In [14]:
rev_busi_Pho.describe() 

,stars
count,219559.000000
mean,3.889233
std,1.220172
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


## Output the filtered csv

In [15]:
rev_busi_Pho.to_csv('../data/filtered_reviews_in_Phonex.csv')